In [1]:
import logging
from pathlib import Path
from Pegasus.api import *
import glob

In [2]:
logging.basicConfig(level=logging.DEBUG)

In [3]:
separate = Transformation(
                "separate",
                site="local",
                pfn="/home/scitech/shared-data/wf-3/separate.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

count = Transformation(
                "count",
                site="local",
                pfn="/home/scitech/shared-data/wf-3/count.py",
                is_stageable=True,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

tar = Transformation(
                "tar",
                site="condorpool",
                pfn="/bin/tar",
                is_stageable=False,
                arch=Arch.X86_64,
                os_type=OS.LINUX
            )

In [4]:
tc = TransformationCatalog().add_transformations(separate, count, tar)

In [5]:
wf = Workflow("workflow3")

In [6]:
file_list = []
rc = ReplicaCatalog()

for f in glob.glob("nums*.txt"):
    file_list.append(File(f))
    rc.add_replica("local", File(f), Path(".").resolve() / f)

fb1 = File("odd_nums.txt")
fb2 = File("even_nums.txt")
fc1 = File("odd_count.txt")
fc2 = File("even_count.txt")

rc.add_replica("local", fb1, Path(".").resolve() / "odd_nums.txt")
rc.add_replica("local", fb2, Path(".").resolve() / "even_nums.txt")

job_separate = Job(separate)\
                    .add_inputs(*file_list)\
                    .add_outputs(fb1, fb2)

job_count_odd = Job(count)\
                .add_args(fb1, fc1)\
                .add_inputs(fb1)\
                .add_outputs(fc1)

job_count_even = Job(count)\
                .add_args(fb2, fc2)\
                .add_inputs(fb2)\
                .add_outputs(fc2)

fc3 = File("result.tar.gz")

job_tar = Job(tar)\
            .add_args("czf", fc3, fc1, fc2)\
            .add_inputs(fc1, fc2)\
            .add_outputs(fc3)



In [7]:
wf.add_jobs(job_separate, job_count_odd, job_count_even, job_tar)
wf.add_replica_catalog(rc)
wf.add_transformation_catalog(tc)

In [8]:
try:
    wf.plan(submit=True)\
        .wait()\
        .analyze()\
        .statistics()
except PegasusClientError as e:
    print(e.output)

Plan:
2020.07.09 18:13:36.312 UTC:    
2020.07.09 18:13:36.317 UTC:   ----------------------------------------------------------------------- 
2020.07.09 18:13:36.323 UTC:   File for submitting this DAG to HTCondor           : workflow3-0.dag.condor.sub 
2020.07.09 18:13:36.329 UTC:   Log of DAGMan debugging messages                 : workflow3-0.dag.dagman.out 
2020.07.09 18:13:36.336 UTC:   Log of HTCondor library output                     : workflow3-0.dag.lib.out 
2020.07.09 18:13:36.341 UTC:   Log of HTCondor library error messages             : workflow3-0.dag.lib.err 
2020.07.09 18:13:36.347 UTC:   Log of the life of condor_dagman itself          : workflow3-0.dag.dagman.log 
2020.07.09 18:13:36.354 UTC:    
2020.07.09 18:13:36.362 UTC:   -no_submit given, not submitting DAG to HTCondor.  You can do this with: 
2020.07.09 18:13:36.374 UTC:   ----------------------------------------------------------------------- 
2020.07.09 18:13:37.553 UTC:   Your database is compatible with P

[################################------------------]  64.7% ..Failure (Completed: 11, Queued: 0, Running: 0, Failed: 1)
Your database is compatible with Pegasus version: 5.0.0dev

************************************Summary*************************************

 Submit Directory   : /home/scitech/shared-data/wf-3/scitech/pegasus/workflow3/run0011
 Total jobs         :     17 (100.00%)
 # jobs succeeded   :     11 (64.71%)
 # jobs failed      :      1 (5.88%)
 # jobs held        :      0 (0.00%)
 # jobs unsubmitted :      5 (29.41%)

******************************Failed jobs' details******************************

=================================tar_ID0000004==================================

 last state: POST_SCRIPT_FAILED
       site: condorpool
submit file: 00/00/tar_ID0000004.sub
output file: 00/00/tar_ID0000004.out.001
 error file: 00/00/tar_ID0000004.err.001

-------------------------------Task #1 - Summary--------------------------------

site        : condorpool
hostname    : 